# Robust SVM (WORK IN PROGRESS)

Support Vector Machines (SVMs) are a set of supervised learning methods primarily used for classification. A Robust SVM is a variation of the standard SVM designed to handle data that is noisy or contains outliers more effectively. While the standard SVM focuses on finding a hyperplane that best separates classes by maximizing the margin between them, a Robust SVM aims to improve the model’s performance and stability in the presence of noise and outliers.

In this notebook, we will consider example 4.5 introduced in Ali, Kolter, Diamond, and Boyd (2015) [1]. Based on the data given, we can reformulate the optimization problem as:

$$
\begin{aligned}
& \text{minimize} \quad \frac{1}{2} \| w \|^2 + C \cdot \vec{1}^T \vec{\xi} \\
& \text{subject to:} \\
& w^T(x_i + u_x) + b \geq 1 - \xi _i\\
&  w^T (x_i + u_y) + b \geq 1 - \xi_i, \\
& \vec{\xi} \geq \vec{0}

\end{aligned}
$$

In [1]:
import numpy as np
import cvxpy as cp
import lropt
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
from scipy.sparse import SparseEfficiencyWarning
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='cvxpy')
warnings.filterwarnings('ignore', category=SparseEfficiencyWarning)

First, we define our data. We have two datasets, both which are normally distributed but have different mean and variance. $N$ is the number of dimensions in the data, which we define as $2$.

In [2]:
np.random.seed(1)
N=2
M = 100
C = 0.01 #Regularization Constant
V = 2
MU_1 = np.array([0, 0])           # Mean for class 1
SIG_1 = np.array([[1, 0.5],       # Covariance for class 1
                  [0.5, 1]])

MU_2 = np.array([3, 3])           # Mean for class 2
SIG_2 = np.array([[1, -0.7],      # Covariance for class 2
                  [-0.7, 1]])

DATA_1= np.random.multivariate_normal(MU_1, SIG_1, M)
DATA_2= np.random.multivariate_normal(MU_2, SIG_2, M)



Next, we can define a function that solves for the Support Vector Machine. This function checks for an uncertainty set. If it isn't present, the function solves the problem in the deterministic case. We use the ellipsoidal uncertainty set, which can be defined by:

$$ \mathcal{U}_{\text{ellips}} = \{Az+b \ | \ \| z\|_2 \le \rho\} $$

In [3]:
def solve_svm(uncertainty_set=None):
    w = cp.Variable(N)
    b = cp.Variable()
    xi = cp.Variable(N, nonneg=True)
    objective = cp.Minimize(1/V * cp.norm2(w)**V + C * (cp.sum(xi)))
    
    constraints = []

    if uncertainty_set:
        u = lropt.UncertainParameter(N, uncertainty_set=uncertainty_set)
        for i in range(N):
            constraints.append(w@(DATA_1[i] + u) + b >= 1 - xi[i])
            constraints.append(-(w@(DATA_2[i]+ u) + b) >= 1 - xi[i])
            prob = lropt.RobustProblem(objective, constraints)
    else:
        for i in range(N):
            constraints.append(w@(DATA_1[i]) + b >= 1 - xi[i])
            constraints.append(-(w@(DATA_2[i]) + b) >= 1 - xi[i])
            prob = cp.Problem(objective, constraints)

    prob.solve()
    
    return w.value, b.value

Finally, we can solve the problem. Note that the value of 'b' in the uncertainty set is the mean of the problem in the deterministic case. 

In [4]:
v = solve_svm(lropt.Ellipsoidal(p = 2,rho=0.1, b = solve_svm()[0]))
w_opt = v[0]
b_opt = v[1]

This is an interactive visualization of the hyperplane created in Altair.

In [5]:
df = pd.DataFrame({
    'Feature 1': np.vstack([DATA_1, DATA_2])[:, 0],
    'Feature 2': np.vstack([DATA_1, DATA_2])[:, 1],
    'Class': ['Class 1'] * M + ['Class 2'] * M
})

x_range = np.linspace(-5, 7, 100)
y_range = - (w_opt[0] * x_range + b_opt) / w_opt[1]

hyperplane_df = pd.DataFrame({'x': x_range, 'y': y_range})
mean_df = pd.DataFrame({
    'Feature 1': [MU_1[0], MU_2[0]],
    'Feature 2': [MU_1[1], MU_2[1]],
    'Class': ['Class 1', 'Class 2']
})

selection = alt.selection_point(on='mouseover', fields=['Feature 1', 'Feature 2'], nearest=True)

base = alt.Chart(df).encode(
    x='Feature 1',
    y='Feature 2',
    color='Class',
    shape='Class'
)

points = base.mark_point(size=60).encode(
    tooltip=['Feature 1:Q', 'Feature 2:Q', 'Class:N']
).add_params(
    selection
)

highlight = base.mark_point(size=100, filled=True, stroke='black').encode(
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).transform_filter(
    selection
)

hyperplane = alt.Chart(hyperplane_df).mark_line(color='black', strokeDash=[5,5]).encode(
    x='x:Q',
    y='y:Q'
)

means = alt.Chart(mean_df).mark_point(size=100, filled=True).encode(
    x='Feature 1',
    y='Feature 2',
    color='Class',
    shape='Class'
)

chart = points + highlight + hyperplane + means

chart = chart.properties(
    width=600,
    height=400,
    title='Robust Chance Constrained SVM'
).configure_axis(
    grid=False
).configure_view(
    strokeOpacity=0
).interactive() 

chart.show()

alt.LayerChart(...)

## References

1. Ali, A., Kolter, J. Z., Diamond, S., & Boyd, S. (2015). Disciplined convex stochastic programming: A new framework for stochastic optimization. Proceedings of the 31st Conference on Uncertainty in Artificial Intelligence.
